In [1]:
import json
from wwo_hist import retrieve_hist_data
import pandas as pd
import pickle
import sklearn

In [2]:
class GET_DATA:
    def __init__(self):
        self.json_data = json.load(open('update_data.json','r'))
        

        
    def get_data(self,location_name):
        for data in self.json_data:
            if data["location"] in location_name:
                lat = data["latitude"]
                log = data["longitude"]
                den = data["density_per_km"]

        return den, lat, log
    
    def get_wheather_data(self,location_name,start_date,end_date):

        start_date = pd.to_datetime(start_date, format="%Y-%m-%d")
        start_date = start_date - pd.to_timedelta(7, unit='d')
        start_date = start_date.strftime('%Y-%m-%d')

        end_date = pd.to_datetime(end_date, format="%Y-%m-%d")
        end_date = end_date - pd.to_timedelta(7, unit='d')
        end_date = end_date.strftime('%Y-%m-%d')

        frequency = 24
        api_key = '092a2ffeb04645c5ba855736210909' 

        location_list = [location_name]
        hist_weather_data = retrieve_hist_data(api_key,
                                        location_list,
                                        start_date,
                                        end_date,
                                        frequency,
                                        location_label = False,
                                        export_csv=False,
                                        store_df = True)
        data = hist_weather_data[0]

        data[['maxtempC','mintempC', 'totalSnow_cm', 'sunHour', 'moon_illumination', 'DewPointC',
           'WindGustKmph', 'cloudcover', 'humidity', 'precipMM', 'pressure',
           'visibility', 'winddirDegree']] = data[['maxtempC',
           'mintempC', 'totalSnow_cm', 'sunHour', 'moon_illumination', 'DewPointC',
           'WindGustKmph', 'cloudcover', 'humidity', 'precipMM', 'pressure',
           'visibility', 'winddirDegree']].apply(pd.to_numeric)

        return data
    
    
    def data_for_clf(self,weather_data,latlog_data):

        # create new year and month column from datetime
        weather_data['year'] = pd.DatetimeIndex(weather_data['date_time']).year
        weather_data['month'] = pd.DatetimeIndex(weather_data['date_time']).month
        weather_data = weather_data.drop(['uvIndex','location','moonrise', 
                                          'moonset', 'sunrise','sunset','FeelsLikeC', 
                                          'HeatIndexC', 'WindChillC','tempC','windspeedKmph'], axis = 1)

        weather_data['density_per_km'] = latlog_data[0]
        weather_data['latitude'] = latlog_data[1]
        weather_data['longitude'] = latlog_data[2]
        weather_data = weather_data[['date_time', 'density_per_km', 'latitude', 'longitude', 'maxtempC', 
                                     'mintempC', 'totalSnow_cm', 'sunHour','moon_illumination', 'DewPointC',
                                     'WindGustKmph', 'cloudcover','humidity', 'precipMM', 'pressure', 
                                     'visibility', 'winddirDegree','year', 'month']]
        return weather_data
    
    def data_for_rg(self,weather_data,latlog_data):

        # create new year and month column from datetime
        weather_data['year'] = pd.DatetimeIndex(weather_data['date_time']).year
        weather_data['month'] = pd.DatetimeIndex(weather_data['date_time']).month

        weather_data = weather_data.drop(['totalSnow_cm',
           'uvIndex', 'moon_illumination', 'moonrise', 'moonset', 'sunrise',
           'sunset', 'HeatIndexC', 'WindChillC',
           'WindGustKmph', 'pressure','tempC', 'visibility', 'location'], axis = 1)

        weather_data['density_per_km'] = latlog_data[0]
        weather_data['latitude'] = latlog_data[1]
        weather_data['longitude'] = latlog_data[2]
        weather_data = weather_data[['date_time','maxtempC', 'mintempC', 'windspeedKmph', 'year', 'month', 'density_per_km', 'precipMM', 'cloudcover',
                                       'humidity','DewPointC', 'latitude', 'longitude','FeelsLikeC', 'winddirDegree', 'sunHour']]
        return weather_data

In [3]:
model = GET_DATA()
latlog_data = model.get_data("Argentina-Buenos_Aires")
weather_data = model.get_wheather_data('Argentina-Buenos_Aires','2016-02-04','2016-03-11')
clf_data = model.data_for_clf(weather_data,latlog_data)
reg_data = model.data_for_rg(weather_data,latlog_data)



Retrieving weather data for Argentina-Buenos_Aires


Currently retrieving data for Argentina-Buenos_Aires: from 2016-01-28 to 2016-01-31
Time elapsed (hh:mm:ss.ms) 0:00:00.684912
Currently retrieving data for Argentina-Buenos_Aires: from 2016-02-01 to 2016-02-29
Time elapsed (hh:mm:ss.ms) 0:00:01.621318
Currently retrieving data for Argentina-Buenos_Aires: from 2016-03-01 to 2016-03-04
Time elapsed (hh:mm:ss.ms) 0:00:02.020079


,date_time,maxtempC,mintempC,windspeedKmph,year,month,density_per_km,precipMM,cloudcover,humidity,DewPointC,latitude,longitude,FeelsLikeC,winddirDegree,sunHour
0,2016-01-28,28,15,17,2016,1,12625,0.2,28,49,9,-34.603684,-58.381559,21,177,11.6
0,2016-01-29,32,15,9,2016,1,12625,1.0,20,52,11,-34.603684,-58.381559,22,176,10.5
0,2016-01-30,30,17,17,2016,1,12625,2.5,19,65,14,-34.603684,-58.381559,23,160,11.6
0,2016-01-31,25,10,20,2016,1,12625,0.0,14,63,9,-34.603684,-58.381559,17,167,11.6
0,2016-02-01,28,11,13,2016,2,12625,0.0,20,65,10,-34.603684,-58.381559,18,130,11.6
0,2016-02-02,30,13,11,2016,2,12625,0.0,10,59,11,-34.603684,-58.381559,21,42,11.6
0,2016-02-03,33,15,16,2016,2,12625,0.0,29,55,12,-34.603684,-58.381559,23,70,11.6
0,2016-02-04,33,17,13,2016,2,12625,0.3,25,50,12,-34.603684,-58.381559,25,132,10.4
0,2016-02-05,31,18,11,2016,2,12625,3.6,34,60,15,-34.603684,-58.381559,25,184,11.6
0,2016-02-06,31,18,13,2016,2,12625,11.9,45,73,18,-34.603684,-58.381559,24,64,11.6


In [373]:
class clf_Model:
    global classifier
    classifier = pickle.load(open("classifier_model.pkl","rb"))
       
        
    def classifier_model(self,data):
        global classifier
        predict_data = data.drop(['date_time'], axis = 1)
        result = classifier.predict_proba(predict_data)
        nozika = [x[0] for x in result]
        zika = [x[1] for x in result]
        data['Zika'] = zika
        data['No Zika'] = nozika

        return data
    
clf_model = clf_Model()

In [374]:
clf_result = clf_model.classifier_model(clf_data)
clf_result

,date_time,density_per_km,latitude,longitude,maxtempC,mintempC,totalSnow_cm,sunHour,moon_illumination,DewPointC,...,cloudcover,humidity,precipMM,pressure,visibility,winddirDegree,year,month,Zika,No Zika
0,2016-01-28,12625,-34.603684,-58.381559,28,15,0.0,11.6,69,9,...,28,49,0.2,1014,10,177,2016,1,0.287580,0.712420
0,2016-01-29,12625,-34.603684,-58.381559,32,15,0.0,10.5,61,11,...,20,52,1.0,1013,10,176,2016,1,0.289510,0.710490
0,2016-01-30,12625,-34.603684,-58.381559,30,17,0.0,11.6,54,14,...,19,65,2.5,1015,10,160,2016,1,0.318075,0.681925
0,2016-01-31,12625,-34.603684,-58.381559,25,10,0.0,11.6,50,9,...,14,63,0.0,1020,9,167,2016,1,0.198488,0.801512
0,2016-02-01,12625,-34.603684,-58.381559,28,11,0.0,11.6,34,10,...,20,65,0.0,1021,8,130,2016,2,0.251371,0.748629
0,2016-02-02,12625,-34.603684,-58.381559,30,13,0.0,11.6,26,11,...,10,59,0.0,1018,8,42,2016,2,0.234247,0.765753
0,2016-02-03,12625,-34.603684,-58.381559,33,15,0.0,11.6,18,12,...,29,55,0.0,1012,9,70,2016,2,0.297792,0.702208
0,2016-02-04,12625,-34.603684,-58.381559,33,17,0.0,10.4,11,12,...,25,50,0.3,1010,10,132,2016,2,0.299516,0.700484


In [375]:
# from sklearn.ensemble import RandomForestRegressor
class reg_Model:
    global regressor
    regressor = pickle.load(open("regression_model.pkl","rb"))

    def regressor_model(self,predict_data):
        global regressor 
        predict_data = predict_data.drop(['date_time'], axis = 1)
        result = regressor.predict(predict_data)
        return result
    
reg_model = reg_Model()

In [377]:
reg_result = reg_model.regressor_model(reg_data)
reg_result

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 3000 out of 3000 | elapsed:    0.2s finished


array([131.16897993,  96.43905099, 124.9360141 ,  87.51267286,
        65.60643922,  50.62769937,  83.84292667,  94.1773086 ])

In [378]:
from datetime import datetime, timedelta
end_date_datetime = datetime.strptime('2016-02-11', "%Y-%m-%d")
end_date_datetime = end_date_datetime + timedelta(days=1)
end_date_datetime

datetime.datetime(2016, 2, 12, 0, 0)

In [379]:
import datetime

class Display_result:
    def data_for_display(self,prob_data,reg_result,start_date,end_date):
        
        date = []
        start = datetime.datetime.strptime(start_date, "%Y-%m-%d")
        end = datetime.datetime.strptime(end_date, "%Y-%m-%d")
        date_array = (start + datetime.timedelta(days=x) for x in range(0, ((end-start).days)+1))

        for date_object in date_array:
            date.append(date_object.strftime("%Y-%m-%d"))
    
    
        data = prob_data.drop(['density_per_km', 'latitude', 'longitude',
           'totalSnow_cm', 'sunHour', 'moon_illumination',
           'WindGustKmph', 'cloudcover', 'pressure',
           'visibility', 'winddirDegree', 'year', 'month'], axis = 1)


        data['No. of cases'] = reg_result

        mean_of_zika = round(data['Zika'].mean()*100)
        mean_of_nozika = round(data['No Zika'].mean()*100)
        total_no_of_cases = int(data['No. of cases'].mean())

        
        data['Zika'] = pd.Series(["{0:.0f}%".format(val * 100) for val in data['Zika']], index = data.index)
        data['No Zika'] = pd.Series(["{0:.0f}%".format(val * 100) for val in data['No Zika']], index = data.index)
        data['No. of cases'] = pd.Series(["{0:.0f}".format(val) for val in data['No. of cases']], index = data.index)
        data.insert(0, "Date", date)

        data.rename(columns={'date_time': 'Climate date', 'maxtempC': 'Max temp','mintempC': 'Min temp','DewPointC': 'Dew point', 'humidity': 'Humdity', 'precipMM': 'Precipitation'}, inplace=True)
        data = data.reset_index()
        data = data.drop(['index'],axis=1)

        return data, mean_of_zika, mean_of_nozika, total_no_of_cases

In [380]:
result = Display_result()

In [381]:
df = result.data_for_display(clf_result,reg_result,'2016-02-04','2016-02-11')
df[0]

,Date,Climate date,Max temp,Min temp,Dew point,Humdity,Precipitation,Zika,No Zika,No. of cases
0,2016-02-04,2016-01-28,28,15,9,49,0.2,29%,71%,131
1,2016-02-05,2016-01-29,32,15,11,52,1.0,29%,71%,96
2,2016-02-06,2016-01-30,30,17,14,65,2.5,32%,68%,125
3,2016-02-07,2016-01-31,25,10,9,63,0.0,20%,80%,88
4,2016-02-08,2016-02-01,28,11,10,65,0.0,25%,75%,66
5,2016-02-09,2016-02-02,30,13,11,59,0.0,23%,77%,51
6,2016-02-10,2016-02-03,33,15,12,55,0.0,30%,70%,84
7,2016-02-11,2016-02-04,33,17,12,50,0.3,30%,70%,94


In [383]:
df[0].to_csv('linechart.csv', index=False)

In [135]:
df[0].columns.values

array(['Date', 'Max temp', 'mintempC', 'Dew point', 'Humdity', 'precipMM',
       'Zika', 'No Zika', 'No. of cases'], dtype=object)

In [136]:
row_data=list(df[0].values.tolist())
row_data

[[Timestamp('2016-01-28 00:00:00'), 28, 15, 9, 49, 0.2, '29%', '71%', '131'],
 [Timestamp('2016-01-29 00:00:00'), 32, 15, 11, 52, 1.0, '29%', '71%', '96'],
 [Timestamp('2016-01-30 00:00:00'), 30, 17, 14, 65, 2.5, '32%', '68%', '125'],
 [Timestamp('2016-01-31 00:00:00'), 25, 10, 9, 63, 0.0, '20%', '80%', '88'],
 [Timestamp('2016-02-01 00:00:00'), 28, 11, 10, 65, 0.0, '25%', '75%', '66'],
 [Timestamp('2016-02-02 00:00:00'), 30, 13, 11, 59, 0.0, '23%', '77%', '51'],
 [Timestamp('2016-02-03 00:00:00'), 33, 15, 12, 55, 0.0, '30%', '70%', '84'],
 [Timestamp('2016-02-04 00:00:00'), 33, 17, 12, 50, 0.3, '30%', '70%', '94']]

In [358]:
import datetime

date = []
start = datetime.datetime.strptime("2016-06-15", "%Y-%m-%d")
end = datetime.datetime.strptime("2016-06-30", "%Y-%m-%d")
date_array = (start + datetime.timedelta(days=x) for x in range(1, (end-start).days))
 
for date_object in date_array:
    date.append(date_object.strftime("%Y-%m-%d"))
    
date

['2016-06-16',
 '2016-06-17',
 '2016-06-18',
 '2016-06-19',
 '2016-06-20',
 '2016-06-21',
 '2016-06-22',
 '2016-06-23',
 '2016-06-24',
 '2016-06-25',
 '2016-06-26',
 '2016-06-27',
 '2016-06-28',
 '2016-06-29']